In [27]:
import logging
import datetime
import os
import json
from dotenv import load_dotenv

#for UI
import streamlit as st

# for function
from function import (connect_openai_llm, generate_answer, display_hits_dataframe)
# from prompt import generate_pdf_prompt

In [28]:
load_dotenv()
OPENAI_API_KEY=  os.getenv("OPENAI_API_KEY")


In [59]:
# json
# instruction = """You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the speficied subject and its content. From the email, please extract the following entity in to jason format provided below
# [{"Yacht Model": ""},{"Yacht Length": ""},{"Year of Manufacture":""}{"Current Value/Purchase Price":""},{"Current Location":""},{"Intended Cruising Area":""},{"Owner's Name":""},{"Owner's Contact Information":""},{"Owner's Boating Experience":""},{"Previous Insurance Claims":""},{"Additional Equipment":""},{"Current Insurance Coverage":""},{"Other":""}]"""

# pandas dataframe
instruction = """You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the specified subject and its content. From the email, please extract the following entities into a list with the order specified below:
["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner's Name", "Owner's Contact Information", "Owner's Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]
"""

# python dictionary
# instruction = """You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the speficied subject and its content. From the email, please extract the following entity in to python dictionary format provided below
# {"Yacht Model": "Sunseeker Manhattan 68","Yacht Length": "68 feet","Year of Manufacture": "2022","Current Value/Purchase Price": "$3,200,000","Current Location": "Marina del Rey, California","Intended Cruising Area": "Coastal cruising, occasional trips to Mexico","Owner's Name": "John Smith","Owner's Contact Information": "(310) 555-1234","Owner's Boating Experience": "10 years, including 5 years with vessels over 50 feet","Previous Insurance Claims": "None","Additional Equipment": "Max speed: 32 knots, Sleeping capacity: 8 people","Current Insurance Coverage": "Looking for comprehensive insurance coverage","Other": "Interested in options for liability coverage"}"""

In [54]:
def generate_prompt(email):
    messages = {
        "messages": [
            {
            "content": instruction,
            "role": "system"
            },
            {
            "content": f"""INSURANCE REQUEST EMAIL: {email}""",
            "role": "user"
            },
            {
            "content": "",
            "role": "assistant"
            },
        ]
    }
    return messages["messages"]

In [31]:
customer_email= """Subject: Insurance Quote for My New Sunseeker

To whom it may concern,

I recently purchased a Sunseeker Manhattan 68 yacht and I'm looking for comprehensive insurance coverage. The yacht is 68 feet long and was built in 2022. It's currently docked at Marina del Rey in California.

Some additional details:

- Purchase price: $3,200,000
- Max speed: 32 knots
- Sleeping capacity: 8 people
- Planning to use it mainly for coastal cruising, with occasional trips to Mexico

I have 10 years of boating experience, including 5 years with vessels over 50 feet. No accidents or insurance claims in the past.

Could you please provide me with a quote for full coverage? I'm particularly interested in options for liability coverage.

Thank you,
John Smith
Phone: (310) 555-1234"""

In [32]:
model_id_llm='gpt-4o'
model_id_emb="text-embedding-3-large"
model_llm = connect_openai_llm()

In [60]:
prompt = generate_prompt(customer_email)
print(prompt)
response = generate_answer(model_llm,prompt)
print(response)
st.text_area(label="Model Response", value=response, height=300)
# display_hits_dataframe(hits)

[{'content': 'You are a helpful, respectful assistant. You will receive an INSURANCE REQUEST EMAIL in the specified subject and its content. From the email, please extract the following entities into a list with the order specified below:\n["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner\'s Name", "Owner\'s Contact Information", "Owner\'s Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]\n', 'role': 'system'}, {'content': "INSURANCE REQUEST EMAIL: Subject: Insurance Quote for My New Sunseeker\n\nTo whom it may concern,\n\nI recently purchased a Sunseeker Manhattan 68 yacht and I'm looking for comprehensive insurance coverage. The yacht is 68 feet long and was built in 2022. It's currently docked at Marina del Rey in California.\n\nSome additional details:\n\n- Purchase price: $3,200,000\n- Max speed: 32 knots\n- Sleeping capacity: 8 p

'["Sunseeker Manhattan 68", "68 feet", "2022", "$3,200,000", "Marina del Rey, California", "Coastal cruising, with occasional trips to Mexico", "John Smith", "(310) 555-1234", "10 years of boating experience, including 5 years with vessels over 50 feet", "No accidents or insurance claims in the past", "Max speed: 32 knots, Sleeping capacity: 8 people", "Not specified", "Request for full coverage, interested in liability options"]'

In [39]:
response_dic = response

In [50]:
response_json = response.replace('\n', '').replace('json', '').replace('```', '').replace('', '')
response_json

'[  {"Yacht Model": "Sunseeker Manhattan 68"},  {"Yacht Length": "68 feet"},  {"Year of Manufacture":"2022"},  {"Current Value/Purchase Price":"$3,200,000"},  {"Current Location":"Marina del Rey, California"},  {"Intended Cruising Area":"Coastal cruising, occasional trips to Mexico"},  {"Owner\'s Name":"John Smith"},  {"Owner\'s Contact Information":"Phone: (310) 555-1234"},  {"Owner\'s Boating Experience":"10 years of boating experience, including 5 years with vessels over 50 feet"},  {"Previous Insurance Claims":"No accidents or insurance claims in the past"},  {"Additional Equipment":""},  {"Current Insurance Coverage":""},  {"Other":"Max speed: 32 knots, Sleeping capacity: 8 people"}]'

In [52]:
data = json.loads(response_json)
data

[{'Yacht Model': 'Sunseeker Manhattan 68'},
 {'Yacht Length': '68 feet'},
 {'Year of Manufacture': '2022'},
 {'Current Value/Purchase Price': '$3,200,000'},
 {'Current Location': 'Marina del Rey, California'},
 {'Intended Cruising Area': 'Coastal cruising, occasional trips to Mexico'},
 {"Owner's Name": 'John Smith'},
 {"Owner's Contact Information": 'Phone: (310) 555-1234'},
 {"Owner's Boating Experience": '10 years of boating experience, including 5 years with vessels over 50 feet'},
 {'Previous Insurance Claims': 'No accidents or insurance claims in the past'},
 {'Additional Equipment': ''},
 {'Current Insurance Coverage': ''},
 {'Other': 'Max speed: 32 knots, Sleeping capacity: 8 people'}]

dataframe pd

In [56]:
import pandas as pd

In [64]:
col = ["Yacht Model", "Yacht Length", "Year of Manufacture", "Current Value/Purchase Price", "Current Location", "Intended Cruising Area", "Owner's Name", "Owner's Contact Information", "Owner's Boating Experience", "Previous Insurance Claims", "Additional Equipment", "Current Insurance Coverage", "Other"]
df = pd.DataFrame(index=col)
df

""
Yacht Model
Yacht Length
Year of Manufacture
Current Value/Purchase Price
Current Location
Intended Cruising Area
Owner's Name
Owner's Contact Information
Owner's Boating Experience
Previous Insurance Claims


In [63]:
#str > list > Series > into df
import ast
response_list = ast.literal_eval(response)

In [65]:
df['new']=response_list

In [66]:
df

,new
Yacht Model,Sunseeker Manhattan 68
Yacht Length,68 feet
Year of Manufacture,2022
Current Value/Purchase Price,"$3,200,000"
Current Location,"Marina del Rey, California"
Intended Cruising Area,"Coastal cruising, with occasional trips to Mexico"
Owner's Name,John Smith
Owner's Contact Information,(310) 555-1234
Owner's Boating Experience,"10 years of boating experience, including 5 ye..."
Previous Insurance Claims,No accidents or insurance claims in the past
